# I altered 100-02 in train2 and replaced one of the ***,JR. with *** JR.

# after moving files to the cluster one have to do:
## scl enable rh-python35 bash
## sbatch run.sh

In [2]:
setnumber = 13

In [3]:
import xml.etree.ElementTree
import os
import numpy as np
import random
import glob
import sys

sys.path.append(os.path.join('..','..','src'))
from conll_to_brat import output_entities
import utils
import shutil

In [4]:
assert(not os.path.exists("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/DICT"))
os.mkdir("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/DICT")
assert(not os.path.exists("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/xmltrain"))
os.mkdir("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/xmltrain")
assert(not os.path.exists("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/xmlvalid"))
os.mkdir("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/xmlvalid")

In [5]:
FAllin = np.load("/home/salman/NeuroNER-master/data/FAllin.npy")
SAllin = np.load("/home/salman/NeuroNER-master/data/SAllin.npy")
MAllin = np.load("/home/salman/NeuroNER-master/data/MAllin.npy")

In [6]:
with open("/home/salman/NeuroNER-master/data/Speedi/femalegivenname.txt", 'r') as file :
    femaleDic = file.read().split('\n')
with open("/home/salman/NeuroNER-master/data/Speedi/malegivenname.txt", 'r') as file :
    maleDic = file.read().split('\n')

In [7]:
with open("/home/salman/NeuroNER-master/data/Speedi/last_names_unambig.txt") as f:
    FamiliList=f.read().split()
with open("/home/salman/NeuroNER-master/data/Speedi/family_names.txt") as f:
    FullFamiliList=f.read().split()

In [8]:
MaleOutList = []
FemaleOutList = []
SureOutList = []
for i in femaleDic:
    if i.lower() not in FAllin:
        FemaleOutList.append(i.title())
for i in maleDic:
    if i.lower() not in MAllin:
        MaleOutList.append(i.title())
for i in FullFamiliList:
    if i.lower() not in FAllin:
        SureOutList.append(i.title())

In [9]:
def GetAndReturn(PATIENT,TFlag):
    if len(TFlag) == 2:
        rgFlag = TFlag[0]
        rsFlag = TFlag[1]
    else:
        raise ValueError("TFlag should be with len 2")
    #jr = Givenname = Surename = NewGivenName = NewSureName = ""
    jr = ""
    Givenname = ""
    Surename = ""
    NewGivenName = ""
    NewSureName = ""
    #
    if ((len(PATIENT.split()) == 1) and (len(PATIENT.split(',')) == 1)):
        if PATIENT.lower() in SAllin:
            Surename = PATIENT
            NewSureName = ReturnSure(PATIENT,rsFlag)
        elif ((PATIENT.lower() in FAllin) or (PATIENT.lower() in MAllin)):
            Givenname = PATIENT
            NewGivenName = ReturnGiven(PATIENT,rgFlag)
        elif PATIENT.lower()[-2:]=="\'s":
            if PATIENT.lower()[:-2] in SAllin:
                Surename = PATIENT[:-2]
                NewSureName = ReturnSure(PATIENT[:-2],rsFlag)
            elif ((PATIENT.lower()[:-2] in FAllin) or (PATIENT.lower()[:-2] in MAllin)):
                Givenname = PATIENT[:-2]
                NewGivenName = ReturnGiven(PATIENT[:-2],rgFlag)
            else:
                raise ValueError("PATIENT(%s) is len 1 splited but does not match!"%PATIENT)
        else:
            raise ValueError("!!PATIENT(%s) is len 1 splited but does not match!"%PATIENT)
    elif ((len(PATIENT.split()) == 1) and (len(PATIENT.split(',')) == 2)):
        Surename = PATIENT.split(',')[0]
        Givenname= PATIENT.split(',')[1]
        if ((len(Surename.split()) == 1) and \
            (len(Givenname.split()) == 1) and \
            (Givenname.lower() not in ["jr","jr."])):
            if ((Givenname.lower() in FAllin) or (Givenname.lower() in MAllin)):
                NewGivenName = ReturnGiven(Givenname,rgFlag)
            else:
                raise ValueError("Givenname(%s) not in the list"%Givenname)
            if (Surename.lower() in SAllin):
                NewSureName = ReturnSure(Surename,rsFlag)
            else:
                raise ValueError("Surename(%s) not in the list"%Surename)
        else:
             raise ValueError("Len Surename(%s) and Givenname(%s) are off"%(Surename,Givenname))   

    elif len(PATIENT.split()) == 1:
        raise ValueError("Len of split with space and , are not the same in PATIENT=%s"%PATIENT)
    ######
    elif len(PATIENT.split())==3:
        if (PATIENT.split()[0][-1] == ","):
            Surename = PATIENT.split()[0][:-1]
            if len(PATIENT.split()[1]) == 1:
                jr = PATIENT.split()[1]
                Givenname = PATIENT.split()[2]
            elif PATIENT.split()[1][-1] == ".":
                jr = PATIENT.split()[1][:-1]
                Givenname = PATIENT.split()[2]
            elif PATIENT.split()[1][-2] == ".":
                jr = PATIENT.split()[1][:-2]
                Givenname = PATIENT.split()[2]
            elif PATIENT.split()[1].lower() == "jr":
                jr = PATIENT.split()[1]
                Givenname = PATIENT.split()[2]
            elif len(PATIENT.split()[2]) == 1:
                jr = PATIENT.split()[2]
                Givenname = PATIENT.split()[1]
            elif PATIENT.split()[2][-1] == ".":
                jr = PATIENT.split()[2][:-1]
                Givenname = PATIENT.split()[1]
            elif PATIENT.split()[2][-2] == ".":
                jr = PATIENT.split()[2][:-2]
                Givenname = PATIENT.split()[1]
            elif PATIENT.split()[2].lower() == "jr":
                jr = PATIENT.split()[2]
                Givenname = PATIENT.split()[1]
            else:
                raise ValueError("PATIENT(%s) first part has , but the rest is not well defined"%PATIENT)
        else:
            if len(PATIENT.split()[0]) == 1:
                jr = PATIENT.split()[0]
                Givenname = PATIENT.split()[1]
                Surename = PATIENT.split()[2]
            elif PATIENT.split()[0][-1] == ".":
                jr = PATIENT.split()[0][:-1]
                Givenname = PATIENT.split()[1]
                Surename = PATIENT.split()[2]
            elif PATIENT.split()[0][-2] == ".":
                jr = PATIENT.split()[0][:-2]
                Givenname = PATIENT.split()[1]
                Surename = PATIENT.split()[2]
            elif PATIENT.split()[0].lower() == "jr":
                jr = PATIENT.split()[0]
                Givenname = PATIENT.split()[1]
                Surename = PATIENT.split()[2]
            else:
                if len(PATIENT.split()[1]) == 1:
                    jr = PATIENT.split()[1]
                    Givenname = PATIENT.split()[0]
                    Surename = PATIENT.split()[2]
                elif PATIENT.split()[1][-1] == ".":
                    jr = PATIENT.split()[1][:-1]
                    Givenname = PATIENT.split()[0]
                    Surename = PATIENT.split()[2]
                elif PATIENT.split()[1][-2] == ".":
                    jr = PATIENT.split()[1][:-2]
                    Givenname = PATIENT.split()[0]
                    Surename = PATIENT.split()[2]
                elif PATIENT.split()[1].lower() == "jr":
                    jr = PATIENT.split()[1]
                    Givenname = PATIENT.split()[0]
                    Surename = PATIENT.split()[2]
                else:
                    if len(PATIENT.split()[2]) == 1:
                        jr = PATIENT.split()[2]
                        Givenname = PATIENT.split()[0]
                        Surename = PATIENT.split()[1]
                    elif PATIENT.split()[2][-1] == ".":
                        jr = PATIENT.split()[2][:-1]
                        Givenname = PATIENT.split()[0]
                        Surename = PATIENT.split()[1]
                    elif PATIENT.split()[2][-2] == ".":
                        jr = PATIENT.split()[2][:-2]
                        Givenname = PATIENT.split()[0]
                        Surename = PATIENT.split()[1]
                    elif PATIENT.split()[2].lower() == "jr":
                        jr = PATIENT.split()[2]
                        Givenname = PATIENT.split()[0]
                        Surename = PATIENT.split()[1]
                    else:
                        raise ValueError("PATIENT(%s) is not well defined"%PATIENT)
        if ((Givenname.lower() in FAllin) or (Givenname.lower() in MAllin)):
            NewGivenName = ReturnGiven(Givenname,rgFlag)
        else:
            raise ValueError("Givenname(%s) not in the list"%Givenname)
        if (Surename.lower() in SAllin):
            NewSureName = ReturnSure(Surename,rsFlag)
        else:
            raise ValueError("Surename(%s) not in the list"%Surename)
    ######
    elif len(PATIENT.split(','))==3:
        raise ValueError("PATIENT(%s) , gives 3 but space does not."%PATIENT)
    ######
    elif len(PATIENT.split())==2:
        if PATIENT.split()[0][-1] == ",":
            Surename = PATIENT.split()[0][:-1]
            Givenname= PATIENT.split()[1]
            if ((len(Surename.upper().split()) == 1) and \
                (len(Givenname.split()) == 1) and \
                (Givenname.lower() not in ["jr","jr."])):
                if PATIENT.split(",")[0] == Surename:
                    if ((Givenname.lower() in FAllin) or (Givenname.lower() in MAllin)):
                        NewGivenName = ReturnGiven(Givenname,rgFlag)
                    else:
                        raise ValueError("From Patinet(%s), Givenname(%s) not in the list"%(PATIENT,Givenname))
                    if (Surename.lower() in SAllin):
                        NewSureName = ReturnSure(Surename,rsFlag)
                    else:
                        raise ValueError("!Surename(%s) not in the list"%Surename)
                else:
                    raise ValueError("Surename = %s but PATIENT.split , 0 gives different(PATIENT=%s)"%(Surename,PATIENT))
            elif Givenname.lower() in ["jr","jr."]:
                NewGivenName = Givenname
            else:
                raise ValueError("len of split of Givenname(%s) or Surename(%s) is not 1!"%(Givenname,Surename))
        elif ((len(PATIENT.split()[0].split(",")) == 1) and \
              (len(PATIENT.split()[1].split(",")) == 1)):
            Givenname= PATIENT.split()[0]
            Surename = PATIENT.split()[1]
            if ((Givenname.lower() in FAllin) or (Givenname.lower() in MAllin)):
                NewGivenName = ReturnGiven(Givenname,rgFlag)
            elif Givenname.lower() in ["jr","jr."]:
                NewGivenName = Givenname
            else:
                raise ValueError("Givenname(%s) not in the list. The PATIENT is %s"%(Givenname,PATIENT))
            if (Surename.lower() in SAllin):
                NewSureName = ReturnSure(Surename,rsFlag)
            else:
                raise ValueError("Surename(%s) not in the list. The PATIENT is %s"%(Surename,PATIENT))
        elif((PATIENT.split(",")[1].split()[1].lower() in ["jr","jr.","j."]) or \
             len(PATIENT.split(",")[1].split()[1]) == 1 or \
             PATIENT.split(",")[1].split()[1] == "FRANCIS" or \
             PATIENT.split(",")[1].split()[1] == "E."):##THIS IS A HACK for file 297-05.txt!
            jr=PATIENT.split(",")[1].split()[1]
            Givenname = PATIENT.split(",")[1].split()[0]
            Surename = PATIENT.split(",")[0]
            if ((len(Surename.split()) == 1) and \
                (len(Givenname.split()) == 1) and \
                (Givenname.lower() not in ["jr","jr."])):
                if ((Givenname.lower() in FAllin) or (Givenname.lower() in MAllin)):
                    NewGivenName = ReturnGiven(Givenname,rgFlag)
                else:
                    raise ValueError("Givenname(%s) not in the list"%Givenname)
                if (Surename.lower() in SAllin):
                    NewSureName = ReturnSure(Surename,rsFlag)
                else:
                    raise ValueError("Surename(%s) not in the list"%Surename)
            else:
                raise ValueError("len of split of Givenname(%s) or Surename(%s) is not 1!"%(Givenname,Surename))
        else:
            raise ValueError("This is PATIENT with len 2 that does not fit: %s"%PATIENT)
    return(Givenname,Surename,NewGivenName,NewSureName,jr)

In [10]:
def ReturnGiven(GN,Flag=1):
    if Flag == 0:
        return(GN)
    elif Flag == 1:
        rc = GN.lower()
        if GN.lower() in FAllin:
            while ((rc == GN.lower()) or (rc == ",") or (rc == "*")):
                rc = random.choice(FAllin)
        if GN.lower() in MAllin:
            while ((rc == GN.lower()) or (rc == ",") or (rc == "*")):
                rc = random.choice(MAllin)
        if GN.lower()==GN:
            return(rc)
        if GN.title()==GN:
            return(rc.title())
        if GN.upper()==GN:
            return(rc.upper())
        raise ValueError("Givenname(%s) is not in the gender list"%GN)
    elif Flag == 2:
        rc = GN.lower()
        if GN.lower() in FAllin:
#            while rc.lower() == GN.lower():
            while (rc.lower() == GN.lower() or len(rc.split('-')) > 1 ):
                rc = random.choice(FemaleOutList)
        if GN.lower() in MAllin:
#            while rc.lower() == GN.lower():
            while (rc.lower() == GN.lower() or len(rc.split('-')) > 1 ):
                rc = random.choice(MaleOutList)
        if GN.lower()==GN:
            return(rc.lower())
        if GN.title()==GN:
            return(rc.title())
        if GN.upper()==GN:
            return(rc.upper())
        raise ValueError("Givenname(%s) is not in the gender list"%GN)
    else:
        raise ValueError("Flag should be eather 0, 1 or 2 but is %s"%Flag)

In [11]:
def ReturnSure(SN,Flag=1):
    if Flag == 0:
        return(SN)
    elif Flag == 1:
        rc = SN.lower()
        while rc == SN.lower():
            rc = random.choice(SAllin)
        if SN.lower()==SN:
            return(rc)
        if SN.title()==SN:
            return(rc.title())
        if SN.upper()==SN:
            return(rc.upper())
    elif Flag == 2:
        rc = SN.lower()
        while rc.lower() == SN.lower():
            rc = random.choice(SureOutList)
        if SN.lower()==SN:
            return(rc.lower())
        if SN.title()==SN:
            return(rc.title())
        if SN.upper()==SN:
            return(rc.upper())
        if SN[0].upper()+SN[1:]==SN:
            return(rc[0].upper()+rc[1:])
    else:
        raise ValueError('Flag should be eather 0, 1 or 2 but is %s'%Flag)

In [12]:
input_folder = "training-PHI-Gold-Set1"
output_folder = "xmltrain"
#input_folder = "training-PHI-Gold-Set2"
#output_folder = "xmlvalid"

for input_filepath in sorted(glob.glob(os.path.join(input_folder, '*.xml'))):
    filename = utils.get_basename_without_extension(input_filepath)
    output_text_filepath = os.path.join(output_folder, '{0}.txt'.format(filename))
    tree = xml.etree.ElementTree.parse(input_filepath)
    xmldoc = tree.getroot()
    newtext = str(xmldoc.findtext('TEXT'))
    tags = xmldoc.findall('TAGS')[0] # [0] because there is only one <TAGS>...</TAGS>
    entities = []

    Dict = {}
    Delta = 0
    for n,tag in enumerate(tags):
        entity = {}
        entity['label'] = tag.get('TYPE')
        entity['text'] = tag.get('text')
        entity['start'] = int(tag.get('start'))
        entity['end'] = int(tag.get('end'))
        #
        if entity['label'] == "PATIENT":
            upAD = entity['text']
            inlen = entity['end'] - entity['start']
            assert len(upAD) == inlen, "Oh no! Something is wrong with the length and start/end points"
            Givenname,Surename,NewGivenName,NewSureName,jr = GetAndReturn(tag.get('text'),[2,2])
            if "Givenname" not in Dict:
                Dict["Givenname"] = Givenname
                Dict["NewGivenName"] = NewGivenName
            if "Surename" not in Dict:
                Dict["Surename"] = Surename
                Dict["NewSureName"] = NewSureName
            if "jr" not in Dict:
                Dict["jr"] = jr

            if upAD.find(Givenname)!=-1 and upAD.find(Surename)!=-1:
                if(upAD.find(Givenname) < upAD.find(Surename)):
                    upAD = upAD.replace(Givenname,Dict["NewGivenName"],1)
                    upAD = upAD[::-1].replace(Surename[::-1],Dict["NewSureName"][::-1],1)[::-1]
                elif(upAD.find(Givenname) > upAD.find(Surename)):
                    upAD = upAD.replace(Surename,Dict["NewSureName"],1)
                    upAD = upAD[::-1].replace(Givenname[::-1],Dict["NewGivenName"][::-1],1)[::-1]
            if ((len(Givenname) != 0) and (len(NewGivenName) != 0)):
                upAD = upAD.replace(Givenname,Dict["NewGivenName"])
            if ((len(Surename) != 0) and (len(NewSureName) !=0 )):
                upAD = upAD.replace(Surename,Dict["NewSureName"])
            tag.set('text',upAD)
            newtext = newtext[:Delta + entity['start']] + upAD + newtext[Delta + entity['end']:]
            tag.set('start',str(Delta + entity['start']))
            Delta = Delta + (len(upAD) - inlen)
            #print(Delta)
            #two line up
            tag.set('end',str(Delta + entity['end']))
        else:
            tag.set('start',str(Delta + entity['start']))
            tag.set('end',str(Delta + entity['end']))
        entities.append(entity)
    tree.find('TEXT').text = newtext
    tree.write('%s/%s.xml'%(output_folder,filename),encoding="UTF-8")
    np.save("DICT/%s"%filename,Dict)

In [13]:
a = np.array('harlan')
MAllin = np.append(MAllin,a)
a = np.array('har')
MAllin = np.append(MAllin,a)
a = np.array('valdez')
SAllin = np.append(SAllin,a)
a = np.array('donna')
FAllin = np.append(FAllin,a)
a = np.array('russell')
SAllin = np.append(SAllin,a)
a = np.array('bruce')
MAllin = np.append(MAllin,a)
a = np.array('corona')
SAllin = np.append(SAllin,a)
a = np.array('julia')
FAllin = np.append(FAllin,a)
a = np.array('thomas-yosef')
SAllin = np.append(SAllin,a)
a = np.array('jennifer')
FAllin = np.append(FAllin,a)
a = np.array('booker')
SAllin = np.append(SAllin,a)
a = np.array('sandy')
MAllin = np.append(MAllin,a)
a = np.array('tuttle')
SAllin = np.append(SAllin,a)
a = np.array('samuel')
MAllin = np.append(MAllin,a)
a = np.array('quiana')
FAllin = np.append(FAllin,a)
a = np.array('justus')
SAllin = np.append(SAllin,a)
a = np.array('iyanna')
FAllin = np.append(FAllin,a)
a = np.array('field')
SAllin = np.append(SAllin,a)
a = np.array('percy')
MAllin = np.append(MAllin,a)
a = np.array('root')
SAllin = np.append(SAllin,a)
a = np.array('val')
MAllin = np.append(MAllin,a)
a = np.array('egan')
SAllin = np.append(SAllin,a)
a = np.array('gabrielle')
FAllin = np.append(FAllin,a)
a = np.array('oreilly')
SAllin = np.append(SAllin,a)
a = np.array('teresa')
FAllin = np.append(FAllin,a)
a = np.array('hardy')
SAllin = np.append(SAllin,a)
a = np.array('kenneth')
MAllin = np.append(MAllin,a)
a = np.array('umstead')
SAllin = np.append(SAllin,a)
a = np.array('valeria')
FAllin = np.append(FAllin,a)
a = np.array('nathan-oliver')
MAllin = np.append(MAllin,a)
a = np.array('ocampo')
SAllin = np.append(SAllin,a)
a = np.array('urbina')
FAllin = np.append(FAllin,a)
a = np.array('yvette')
FAllin = np.append(FAllin,a)
a = np.array('ulibarri')
SAllin = np.append(SAllin,a)
a = np.array('edwin')
MAllin = np.append(MAllin,a)
a = np.array('workman')
SAllin = np.append(SAllin,a)
a = np.array('bill')
MAllin = np.append(MAllin,a)
a = np.array('fair')
SAllin = np.append(SAllin,a)
a = np.array('xavier')
MAllin = np.append(MAllin,a)
a = np.array('kaufman')
SAllin = np.append(SAllin,a)
a = np.array('yair')
MAllin = np.append(MAllin,a)
a = np.array('godwin')
SAllin = np.append(SAllin,a)
a = np.array('patricia')
FAllin = np.append(FAllin,a)
a = np.array('yount')
SAllin = np.append(SAllin,a)
a = np.array('sharon')
FAllin = np.append(FAllin,a)
a = np.array('fallon')
SAllin = np.append(SAllin,a)
a = np.array('falon')
SAllin = np.append(SAllin,a)
a = np.array('ruth')
FAllin = np.append(FAllin,a)
a = np.array('xanders')
SAllin = np.append(SAllin,a)
a = np.array('milton')
MAllin = np.append(MAllin,a)
a = np.array('horne')
SAllin = np.append(SAllin,a)
a = np.array('brian')
MAllin = np.append(MAllin,a)
a = np.array('martin')
SAllin = np.append(SAllin,a)
a = np.array('uriel')
MAllin = np.append(MAllin,a)
a = np.array('jones')
SAllin = np.append(SAllin,a)
a = np.array('vastie')
MAllin = np.append(MAllin,a)
a = np.array('uecker')
SAllin = np.append(SAllin,a)
a = np.array('lydia')
FAllin = np.append(FAllin,a)
a = np.array('francis')
SAllin = np.append(SAllin,a)
a = np.array('horn')
SAllin = np.append(SAllin,a)
a = np.array('shari')
FAllin = np.append(FAllin,a)
a = np.array('etheridge')
SAllin = np.append(SAllin,a)
a = np.array('stephen')
MAllin = np.append(MAllin,a)
a = np.array('vines')
SAllin = np.append(SAllin,a)
a = np.array('kelli')
FAllin = np.append(FAllin,a)
a = np.array('westyn-nuel')
SAllin = np.append(SAllin,a)
a = np.array('william-nuel')
SAllin = np.append(SAllin,a)
a = np.array('quinton')
MAllin = np.append(MAllin,a)
a = np.array('welch')
SAllin = np.append(SAllin,a)
a = np.array('quin')
MAllin = np.append(MAllin,a)
a = np.array('quin')
SAllin = np.append(SAllin,a)
a = np.array('tiffany')
FAllin = np.append(FAllin,a)
a = np.array('ulrich')
SAllin = np.append(SAllin,a)
a = np.array('vaccaro')
FAllin = np.append(FAllin,a)
a = np.array('pepper')
MAllin = np.append(MAllin,a)
a = np.array('johnson')
SAllin = np.append(SAllin,a)
a = np.array('alicia')
FAllin = np.append(FAllin,a)
a = np.array('yearwood')
SAllin = np.append(SAllin,a)
a = np.array('roger')
MAllin = np.append(MAllin,a)
a = np.array('wendy')
FAllin = np.append(FAllin,a)
a = np.array('truitt')
SAllin = np.append(SAllin,a)
a = np.array('brenda')
FAllin = np.append(FAllin,a)
a = np.array('brenna')
FAllin = np.append(FAllin,a)
a = np.array('bredna')
FAllin = np.append(FAllin,a)
a = np.array('stephenson')
SAllin = np.append(SAllin,a)
a = np.array('jonathan')
MAllin = np.append(MAllin,a)
a = np.array('oswald')
SAllin = np.append(SAllin,a)
a = np.array('walter')
MAllin = np.append(MAllin,a)
a = np.array('fredrick')
SAllin = np.append(SAllin,a)
a = np.array('fredrick-debra')
SAllin = np.append(SAllin,a)
a = np.array('francisca')
FAllin = np.append(FAllin,a)
a = np.array('duong')
SAllin = np.append(SAllin,a)
a = np.array('cody')
FAllin = np.append(FAllin,a)
a = np.array('codi')
FAllin = np.append(FAllin,a)
a = np.array('youngman')
SAllin = np.append(SAllin,a)
a = np.array('larry')
MAllin = np.append(MAllin,a)
a = np.array('eubanks')
SAllin = np.append(SAllin,a)
a = np.array('john')
MAllin = np.append(MAllin,a)
a = np.array('usrey')
SAllin = np.append(SAllin,a)
a = np.array('quinn')
FAllin = np.append(FAllin,a)
a = np.array('usry')
SAllin = np.append(SAllin,a)
a = np.array('ida')
FAllin = np.append(FAllin,a)
a = np.array('otis')
MAllin = np.append(MAllin,a)
a = np.array('narvaez')
SAllin = np.append(SAllin,a)
a = np.array('oscar')
MAllin = np.append(MAllin,a)
a = np.array('santiago')
FAllin = np.append(FAllin,a)
a = np.array('lloyd')
MAllin = np.append(MAllin,a)
a = np.array('gregg')
SAllin = np.append(SAllin,a)
a = np.array('emily')
FAllin = np.append(FAllin,a)
a = np.array('ippolito')
SAllin = np.append(SAllin,a)
a = np.array('xiao')
MAllin = np.append(MAllin,a)
a = np.array('shields')
MAllin = np.append(MAllin,a)
a = np.array('jennifer')
FAllin = np.append(FAllin,a)
a = np.array('neill')
SAllin = np.append(SAllin,a)
a = np.array('verner')
FAllin = np.append(FAllin,a)
a = np.array('gilberto')
MAllin = np.append(MAllin,a)
a = np.array('ito')
SAllin = np.append(SAllin,a)
a = np.array('iot')
SAllin = np.append(SAllin,a)
a = np.array('george')
MAllin = np.append(MAllin,a)
a = np.array('gustafson')
SAllin = np.append(SAllin,a)
a = np.array('sherman')
MAllin = np.append(MAllin,a)
a = np.array('lagnston')
SAllin = np.append(SAllin,a)
a = np.array('langston')
SAllin = np.append(SAllin,a)
a = np.array('tinsley')
SAllin = np.append(SAllin,a)
a = np.array('levine')
SAllin = np.append(SAllin,a)
a = np.array('levin')
SAllin = np.append(SAllin,a)
a = np.array('daniel')
MAllin = np.append(MAllin,a)
a = np.array('doherty')
SAllin = np.append(SAllin,a)
a = np.array('roberta')
FAllin = np.append(FAllin,a)
a = np.array('vincent')
SAllin = np.append(SAllin,a)
a = np.array('matthew')
MAllin = np.append(MAllin,a)
a = np.array('hart')
SAllin = np.append(SAllin,a)
a = np.array('edith')
FAllin = np.append(FAllin,a)
a = np.array('vandiver')
SAllin = np.append(SAllin,a)
a = np.array('caldwell')
SAllin = np.append(SAllin,a)
a = np.array('wendell')
MAllin = np.append(MAllin,a)
a = np.array('craft')
SAllin = np.append(SAllin,a)
a = np.array('coleman')
SAllin = np.append(SAllin,a)
a = np.array('lara')
SAllin = np.append(SAllin,a)
a = np.array('bell')
SAllin = np.append(SAllin,a)


In [14]:
#input_folder = "training-PHI-Gold-Set1"
#output_folder = "xmltrain"
input_folder = "training-PHI-Gold-Set2"
output_folder = "xmlvalid"

for input_filepath in sorted(glob.glob(os.path.join(input_folder, '*.xml'))):
    filename = utils.get_basename_without_extension(input_filepath)
    output_text_filepath = os.path.join(output_folder, '{0}.txt'.format(filename))
    tree = xml.etree.ElementTree.parse(input_filepath)
    xmldoc = tree.getroot()
    newtext = str(xmldoc.findtext('TEXT'))
    tags = xmldoc.findall('TAGS')[0] # [0] because there is only one <TAGS>...</TAGS>
    entities = []

    Dict = {}
    Delta = 0
    for n,tag in enumerate(tags):
        entity = {}
        entity['label'] = tag.get('TYPE')
        entity['text'] = tag.get('text')
        entity['start'] = int(tag.get('start'))
        entity['end'] = int(tag.get('end'))
        #
        if entity['label'] == "PATIENT":
            upAD = entity['text']
            inlen = entity['end'] - entity['start']
            assert len(upAD) == inlen, "Oh no! Something is wrong with the length and start/end points"
            Givenname,Surename,NewGivenName,NewSureName,jr = GetAndReturn(tag.get('text'),[2,2])
            if "Givenname" not in Dict:
                Dict["Givenname"] = Givenname
                Dict["NewGivenName"] = NewGivenName
            if "Surename" not in Dict:
                Dict["Surename"] = Surename
                Dict["NewSureName"] = NewSureName
            if "jr" not in Dict:
                Dict["jr"] = jr

            if upAD.find(Givenname)!=-1 and upAD.find(Surename)!=-1:
                if(upAD.find(Givenname) < upAD.find(Surename)):
                    upAD = upAD.replace(Givenname,Dict["NewGivenName"],1)
                    upAD = upAD[::-1].replace(Surename[::-1],Dict["NewSureName"][::-1],1)[::-1]
                elif(upAD.find(Givenname) > upAD.find(Surename)):
                    upAD = upAD.replace(Surename,Dict["NewSureName"],1)
                    upAD = upAD[::-1].replace(Givenname[::-1],Dict["NewGivenName"][::-1],1)[::-1]
            if ((len(Givenname) != 0) and (len(NewGivenName) != 0)):
                upAD = upAD.replace(Givenname,Dict["NewGivenName"])
            if ((len(Surename) != 0) and (len(NewSureName) !=0 )):
                upAD = upAD.replace(Surename,Dict["NewSureName"])
            tag.set('text',upAD)
            newtext = newtext[:Delta + entity['start']] + upAD + newtext[Delta + entity['end']:]
            tag.set('start',str(Delta + entity['start']))
            Delta = Delta + (len(upAD) - inlen)
            #print(Delta)
            #two line up
            tag.set('end',str(Delta + entity['end']))
        else:
            tag.set('start',str(Delta + entity['start']))
            tag.set('end',str(Delta + entity['end']))
        entities.append(entity)
    tree.find('TEXT').text = newtext
    tree.write('%s/%s.xml'%(output_folder,filename),encoding="UTF-8")
    np.save("DICT/%s"%filename,Dict)

In [15]:
from xml_to_brat import xml_to_brat

In [16]:
xml_to_brat("xmltrain","train")
xml_to_brat("xmlvalid","valid")

input_folder: xmltrain
Time spent formatting: 0.21 seconds
input_folder: xmlvalid
Time spent formatting: 0.10 seconds


In [17]:
assert(not os.path.exists("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/set%s"%setnumber))
assert(os.path.isfile("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/train/220-01.ann"))
assert(os.path.isfile("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/valid/100-01.ann"))
os.mkdir("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/set%s"%setnumber)
shutil.move("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/train",\
           "/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/set%s"%setnumber)
shutil.move("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/valid",\
           "/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/set%s"%setnumber)
shutil.move("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/DICT",\
           "/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/set%s"%setnumber)
shutil.move("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/xmltrain",\
           "/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/set%s"%setnumber)
shutil.move("/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/xmlvalid",\
           "/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/set%s"%setnumber)

'/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid/set13/xmlvalid'

# after moving 
## scp -r set12 BMIClu:/home/sseyedi/Work/vmNeuroner/runset12/.
# files to the cluster one have to do:
## scl enable rh-python35 bash
## sbatch run.sh (edit it then use it)


In [21]:
pwd

'/home/salman/NeuroNER-master/data/changed_i2b2_2014_deid'

In [ ]:
in /home/sseyedi/Work/vmCheckNeuroner/test1 read runcode.txt
then go to /home/sseyedi/Work/vm2CheckNeuroner/TEST/trained_shadows and do:
    cp -r ../../../vmCheckNeuroner/test1/trained_models/i2b2_shadow12 .


In [19]:
len(FemaleOutList),len(MaleOutList,len(SureOutList)

(4939, 2882, 88845)

In [20]:
len(FAllin),len(MAllin),len(SAllin)

(337, 314, 577)

In [ ]:
update pretrained_model_folder = ./trained_shadows/i2b2_shadow1
update data/i2b2_2014_deid/
run neuroner --parameters_filepath parameters.ini
mv npy files 
rm -r output